# Overlay rails map with area of interest

Notes on the rail buffering:
- round style prevents breaks in the buffers
- USA uses standard track guage (https://en.wikipedia.org/wiki/Track_gauge_in_the_United_States), which is about 1.5m wide

In [43]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, box, LineString
from shapely import intersection

target_epsg = 6350  # https://epsg.org/crs_6350/NAD83-2011-Conus-Albers.html?sessionkey=nb47agmo4r

In [66]:
fn_aoi = '/Volumes/Extreme SSD/largest_plots/clean_data/aoi.parquet'
fn_rails = '/Volumes/Extreme SSD/largest_plots/raw_data/tl_2023_us_rails.zip'
fn_rail_overlay = '/Volumes/Extreme SSD/largest_plots/clean_data/rails_overlay.parquet'

In [7]:
Aoi = gpd.read_parquet(fn_aoi)
Rails = gpd.read_file(fn_rails).to_crs(epsg=target_epsg)

In [60]:
overlay_list = []
for i in Aoi.index:
    single_aoi = Aoi.loc[[i]]
    
    RailSubset = gpd.sjoin(Rails, single_aoi, how='inner', predicate='intersects')
    RailPolygons = RailSubset.buffer(distance=0.75, cap_style='round')  # transform the relevant train lines, represented as shapely lines, into polygons with width. these will be easier to overlay onto the AOI
    rail_polygon_flat = RailPolygons.unary_union
    rail_overlay = Aoi.aoi[i].intersection(rail_polygon_flat)
    overlay_list.append(rail_overlay)

In [68]:
RailOverlay = gpd.GeoDataFrame({'geometry':overlay_list}).set_geometry('geometry')

In [69]:
RailOverlay.to_parquet(fn_rail_overlay)